<a href="https://colab.research.google.com/github/adelic-matf/UMMP/blob/main/UMMP_lab_4_i_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tablična simpleks metoda


Implementiraćemo tabličnu simpleks metodu, prateći algoritam koji je izložen u materijalu sa predavanja.

## Učitavanje neophodnih biblioteka

S obzirom na to da radimo sa matricama, koristićemo biblioteku NumPy, dok ćemo za lepši prikaz tabela koristiti biblioteku Pandas.

In [1]:
import numpy as np
import pandas as pd


## Definisanje zadatka

Rešavamo problem
$$
(min) \quad c^T x
$$
pri ograničenjima
$$
Ax=b
$$
$$
x\geq 0,
$$
gde su zadati:
- matrica $A$ dimenzije $m\times n$,
- vektor $c$ dužine $n$,
- vektor $b$ dužine $m$,

koje ćemo upisati u promenljive sa istim imenom.
Pretpostavljamo da je zadata jedinična baza dimenzije $m$. Indekse bazisno dopustivih rešenja ćemo upisati u vektor basic_vars. Kao test primer koristićemo primer 8 sa predavanja.

$$
\begin{array}
 \text{(min)}\quad &-x_1-2x_2\\
 \\
  \text{p.o.} \quad  &-x_1+x_2\leq 1\\
     &x_1+x_2\leq 3\\
     &x_1,x_2\geq 0.
    \end{array}
  $$

In [2]:
# Koeficijenti funkcije cilja (u kanonskom obliku)
c = np.array([-1, -2, 0, 0])

# Matrica ograničenja (leva strana)
A = np.array([
    [-1, 1, 1, 0],
    [1, 1, 0, 1]
])

# Desna strana ograničenja
b = np.array([1, 3])

# Početna baznisne promenljive
basic_vars = [3, 4]  #


## Formiranje tabele

Za lakšu implementaciju pivotiranja, kreirajmo funkciju koja će od datih ulaznih podataka formirati simpleks tablicu.

In [3]:
# Simpleks tablica
def make_table(A, b, c, basic_vars):
    m, n = A.shape
    table = np.zeros((m + 1, n + 2))
    table[:-1,0]=basic_vars #prva kolona, sve do pretposlednjeg elementa
    table[:-1,1:-1]=A
    table[:-1,-1]=b
    table[-1,1:-1]=c
    return table



Testirajmo

In [4]:
primer8_T0 = make_table(A, b, c, basic_vars)
print(primer8_T0)

[[ 3. -1.  1.  1.  0.  1.]
 [ 4.  1.  1.  0.  1.  3.]
 [ 0. -1. -2.  0.  0.  0.]]


## Štampanje tabele

Radi lepšeg prikaza tabele, formirajmo funkciju koja će izvršiti štampu nalik radu sa tablicama na papiru:

In [5]:
def print_table(table):
  m,n=table.shape
  var_names = [f"a{i}" for i in range(1, n-1 )]
  header = var_names + ['b']
  df = pd.DataFrame(table[:,1:], columns=header)
  # Promenimo indekse
  indeksi='x'+table[:,0].astype(int).astype(str)
  indeksi[-1]='' #posednji indeks prazan
  df.index=indeksi #upisimo indekse
  display(df.head(m))

Testirajmo

In [6]:
print_table(primer8_T0)


,a1,a2,a3,a4,b
x3,-1.0,1.0,1.0,0.0,1.0
x4,1.0,1.0,0.0,1.0,3.0
,-1.0,-2.0,0.0,0.0,0.0


## Jedna iteracija simpleks metode

Implementiraćemo kao zasebnu funkciju jednu iteraciju simpleks metode. Pre toga, izdvojićemo i objasniti neke funkcije iz NumPy biblioteke koje će nam biti od koristi, a to su:
- np.min(niz) - vraća minimalni element niza
- np.argmin(niz) - vraća indeks prvog pojavljivanja minimalnog elementa niza
- np.where(uslov) - vraća torku sa indeksima onih elemenata logičkog niza uslov koji imaju vrednost True; ako znamo da je uslov niz (a ne matrica), prvi element torke  sadrži indekse koji nam trebaju, tj. niz indeksa možemo učitati sa indeksi= np.where(mask)[0]

Svaka od ovih funkcija je definisana i za više od jednog ulaznog argumenta, ali kako ćemo raditi samo sa nizovima, nećemo imati potrebu da koristimo ostale argumente.





Islutrujmo rad sa ovim funkcijama na nekom jednostavnom primeru. Definišimo np niz:

In [7]:
x=np.array([2,-3,0,-2,-3])

Minimalni indeks niza je

In [8]:
x_min=np.min(x)
print(x_min)

-3


Indeks prvog pojavljivanja minimalne vrednosti možemo dobiti na dva načina: koristeći np.where i np.argmin, kao što je prikazano u sledećim linijama koda.

In [9]:
x_min_inds=np.where(x==x_min)[0] # vraca niz indeksa niza x gde se nalazi minimalna vrednost
print(f"Indeksi niza x na kojima se pojavljuje minimalna vrednost su {x_min_inds}, dok je indeks prvog pojavljivanja minimalnog elementa u nizu x jednak {x_min_inds[0]}.")
#ili
x_min_ind=np.argmin(x) # vraca prvi indeks niza x na kom se pojavljuje minimalni element
print(f"Prvi indeks niza x na kome se pojavljuje minimalna vrednost je {x_min_ind}.")

Indeksi niza x na kojima se pojavljuje minimalna vrednost su [1 4], dok je indeks prvog pojavljivanja minimalnog elementa u nizu x jednak 1.
Prvi indeks niza x na kome se pojavljuje minimalna vrednost je 1.


Implementirajmo sada funkciju koja će odgovarati jednoj iteraciji simpleks algoritma
-  **Korak 1**: Ako su svi koeficijenti $r_q\geq0, \,q\in Q$
 tada je trenutno rešenje optimalno i algoritam se završava. U suprotnom idi na korak 2.
-  **Korak 2**: Izabrati kolonu $j$ za koju je $r_j<0$. Promenljiva $x_j$ postaje bazisna.
- **Korak 3**: Za svaku vrstu $i$ izračunati količnik $b_i/a_{ij}$
 , ali samo ako je $a_{ij}>0$. Ako takvih vrednosti nema, problem nema ograničenje i algoritam se zaustavlja.
Inače, izaberi vrstu $l$  sa najmanjim količnikom. Promenljiva $x_{p_l}$ napušta bazu.
- **Korak 4**: Izvršiti pivotiranje po elementu na preseku reda
$l$ i kolone $j$, i ažurirati celu tabelu. Vratiti se na korak 1.

Korake 2 i 3 moramo dodatno precizirati, odnosno definisati kako ćemo birati  kolone ukoliko postoji više elemenata koji zadovoljavaju zadati uslov.

**Korak 2:** Izbor kolone koja ulazi u bazu:  **prvi indeks** na kom se pojavljuje **minimalna vrednost** niza $r<0$.

**Korak 3:** Izbor kolone koja izlazi u bazu:  **prvi indeks** na kom se pojavljuje minimalna vrednost niza sa elementima $b_i/a_{ij}$ gde je $a_{ij}>0$.

In [10]:
def simp_one_step(tab):
    table = tab.copy()  # Kopiramo tabelu
    m, n = table.shape
    opt = 0

    r = table[-1, 1:-1]  # Koeficijenti funkcije cilja (preskačemo kolone za bazu i b)
    if np.all(r >= 0):
        opt = 1  # Rešenje je optimalno
    else:
        # Određivanje indeksa kolone koja ulazi u bazu
        j = np.argmin(r) + 1 #np.argmin vraca indeks prvog minimalnog elementa
        # Dodajemo 1 jer r počinje od kolone 1
        #print(f"Indeks kolone koja ulazi u bazu je j={j}.")

        aj = table[:-1, j]  # Kolona koja ulazi u bazu, isključujemo poslednju vrstu tabele jer su tu koeficijeti funkcije cilja
        b = table[:-1, -1]
        mask = aj > 0

        if np.any(mask):
           # Određivanje kolone koja izlazi iz baze
            ind_mask=np.where(mask)[0] # trazimo indekse onih elemenata u mask koji imaju vrednost True, tj. indekse pozitivnih elemenata u koloni aj
            ind_min = np.argmin(b[mask] / aj[mask]) #trazimo indeks prvog pojavljivanja najmanje vrednosti u maskiranom nizu (kolicnik se racuna samo za one elemente gde je mask True)
            l = ind_mask[ind_min] # iz niza indeksa maske, uzimamo indeks minimalnog elementa, to je indeks kolone koja izlazi iz baze
            #print(f"Indeks vrste koja izlazi iz baze je l={l+1}, tj. bazna promenljiva {int(table[l, 0])}.")

            # Pivotiranje
            pivot = table[l, j]
            table[l, 1:] = table[l, 1:] / pivot  # Normalizujemo pivot red (osim kolone 0)
            table[l, j] = 1.0  # Precizno postavimo pivot na 1 (u slučaju zaokruživanja)
            table[l, 0] = j     # Ažuriramo baznu promenljivu

            for k in range(m):
                if k != l:
                    faktor = table[k, j]
                    table[k, 1:] -= faktor * table[l, 1:]
                    table[k, j] = 0.0  # Precizno postavimo nulu zbog stabilnosti
        else:
            opt = 2  # Problem je neograničen

    return table, opt




In [11]:
primer8_T1,opt=simp_one_step(primer8_T0)
print_table(primer8_T1)



,a1,a2,a3,a4,b
x2,-1.0,1.0,1.0,0.0,1.0
x4,2.0,0.0,-1.0,1.0,2.0
,-3.0,0.0,2.0,0.0,2.0


## Kompletna simpleks metoda

Sada smo spremni da kreiramo funkciju koja će koristeći simp_one_step(tab) iterativno primenjivati simpleks algoritam.



In [12]:
def tablicni_simpleks(table, stampaj=0, maks_iter=1000):
    optimalno = 0
    iter = 0

    while optimalno == 0:
        if stampaj == 1:
            print(f"T{iter}")
            print_table(table)

        # Jedna iteracija simpleks algoritma
        table, optimalno = simp_one_step(table)
        iter += 1

        # Prekid ako broj iteracija premaši dozvoljeni maksimum
        if iter > maks_iter:
            print(f"Stop. Broj iteracija je premašio maksimalan broj {maks_iter}")
            break

    # Nakon izlaska iz petlje: štampa rešenja
    if optimalno == 1:
        print(f"\nOptimalna vrednost problema je {-1 * table[-1, -1]:.4f}, a postiže se za:")
        m, n = table.shape
        for k in range(m - 1):
            print(f"x{int(table[k, 0])} = {table[k, -1]:.4f},")
        print("Ostale promenljive imaju vrednost jednaku 0.")
        print(f"Algoritam se završio u {iter - 1}. iteraciji.")

    elif optimalno == 2:
        print("Problem je neograničen.")

    # Završna tabela
    print_table(table)



In [13]:
tablicni_simpleks(primer8_T0,1)

T0


,a1,a2,a3,a4,b
x3,-1.0,1.0,1.0,0.0,1.0
x4,1.0,1.0,0.0,1.0,3.0
,-1.0,-2.0,0.0,0.0,0.0


T1


,a1,a2,a3,a4,b
x2,-1.0,1.0,1.0,0.0,1.0
x4,2.0,0.0,-1.0,1.0,2.0
,-3.0,0.0,2.0,0.0,2.0


T2


,a1,a2,a3,a4,b
x2,0.0,1.0,0.5,0.5,2.0
x1,1.0,0.0,-0.5,0.5,1.0
,0.0,0.0,0.5,1.5,5.0



Optimalna vrednost problema je -5.0000, a postiže se za:
x2 = 2.0000,
x1 = 1.0000,
Ostale promenljive imaju vrednost jednaku 0.
Algoritam se završio u 2. iteraciji.


,a1,a2,a3,a4,b
x2,0.0,1.0,0.5,0.5,2.0
x1,1.0,0.0,-0.5,0.5,1.0
,0.0,0.0,0.5,1.5,5.0


# Degenerisano rešenje

Kao test primer iskoristimo primer 10 iz materijala sa predavanja.


$$
\begin{array}
     \text{(min)} \quad  
  -2x_1-3x_2+x_3+12x_4\\
    \\
   \qquad -2x_1-9x_2+x_3+9x_4+x_5=0\\
    \qquad 1/3x_1+x_2-1/3x_3-2x_4+x_6=0\\
    \qquad x_i\geq 0,\quad  i=1,2,\dots,6.
    \end{array}
  $$

In [14]:
# Koeficijenti funkcije cilja (u kanonskom obliku)
c = np.array([-2, -3, 1, 12,0,0])

# Matrica ograničenja (leva strana)
A = np.array([
    [-2, -9, 1, 9,1,0],
    [1/3, 1, -1/3, -2,0,1]
])

# Desna strana ograničenja
b = np.array([0, 0])

# Početna baznisne promenljive
basic_vars = [5, 6]  #

Znamo da u ovom primeru algoritam ulazi u ciklus. U 6. iteraciji dobijamo iste bazisne promenljive kao u početnom rešenju (vidi materijal sa predavanja), pa ćemo ograničiti broj iteracija na 6 i ispisati sve tabele kako bismo proverili rezultat.

In [15]:
primer10_T0 = make_table(A, b, c, basic_vars)
tablicni_simpleks(primer10_T0,1,6)

T0


,a1,a2,a3,a4,a5,a6,b
x5,-2.000000,-9.0,1.000000,9.0,1.0,0.0,0.0
x6,0.333333,1.0,-0.333333,-2.0,0.0,1.0,0.0
,-2.000000,-3.0,1.000000,12.0,0.0,0.0,0.0


T1


,a1,a2,a3,a4,a5,a6,b
x5,1.000000,0.0,-2.000000,-9.0,1.0,9.0,0.0
x2,0.333333,1.0,-0.333333,-2.0,0.0,1.0,0.0
,-1.000000,0.0,0.000000,6.0,0.0,3.0,0.0


T2


...

,a1,a2,a3,a4,a5,a6,b
x3,-2.000000,-9.0,1.0,9.0,1.000000,0.0,0.0
x6,-0.333333,-2.0,0.0,1.0,0.333333,1.0,0.0
,0.000000,6.0,0.0,3.0,-1.000000,0.0,0.0


T6


,a1,a2,a3,a4,a5,a6,b
x5,-2.000000,-9.0,1.000000,9.0,1.0,0.0,0.0
x6,0.333333,1.0,-0.333333,-2.0,0.0,1.0,0.0
,-2.000000,-3.0,1.000000,12.0,0.0,0.0,0.0


Stop. Broj iteracija je premašio maksimalan broj 6


,a1,a2,a3,a4,a5,a6,b
x5,1.000000,0.0,-2.000000,-9.0,1.0,9.0,0.0
x2,0.333333,1.0,-0.333333,-2.0,0.0,1.0,0.0
,-1.000000,0.0,0.000000,6.0,0.0,3.0,0.0


## Blendovo pravilo

Kao što je poznato, Blendovo pravilo koje glasi
"među kandidatima za ulaz, odnosno izlaz iz baze uvek birati kolonu sa minimalnim indeksom"
nam omogućava konačnost simpleks algoritma. Dakle, sada moramo izmeniti pravila koja smo prethodno uveli za korak 2 i korak 3:


**Korak 2**: Izbor kolone koja ulazi u bazu bude:  **minimalni indeks niza** $r$ gde se pojavljuje negativna vrednost.

**Korak 3**: Izbor kolone koja izlazi u bazu bude:  **minimalni indeks na kom se pojavljuje minimalna vrednost** niza sa elementima $b_i/a_{ij}$ gde je $a_{ij}>0$.

In [17]:
def simp_one_step_blend(tab):
    table = tab.copy()  # Kopiramo tabelu
    m, n = table.shape
    opt = 0

    r = table[-1, 1:-1]  # Koeficijenti funkcije cilja (preskačemo bazu i b)
    if np.all(r >= 0):
        opt = 1  # Rešenje je optimalno
    else:
        j=np.where(r<0)[0][0]+1
        # Dodajemo 1 jer r počinje od kolone 1
        #print(f"Indeks kolone koja ulazi u bazu je j={j}.")

        aj = table[:-1, j]  # Kolona koja ulazi u bazu
        b = table[:-1, -1]
        mask = aj > 0

        if np.any(mask):
            ind_mask=np.where(mask)[0] # vraca niz sa indeksima onih elemenata u mask koji imaju vrednost True
            ratios = b[mask] / aj[mask] # svi kolicnici za aj>0
            ratio_min = np.min(ratios) # minimalna vrednost kolicnika

            # Indeksi svih elemenata za koje se postize minimalna vrednost kolicnika
            ind_min_ratio=ind_mask[ratios==ratio_min]

            # Blendovo pravilo
            # U prvoj koloni su upisani indeksi bazisnih promenljivih. Biramo minimalnu vrednost od svih
            # onih indeksa koji za koje se postize minimalna vrednost kolicnika i to je indeks
            # kolone koja izlazi iz baze, tj. indeks promenljive koja izlazi iz baze
            b_ind_min=np.min(table[ind_min_ratio,0])

            # Potrebno je jos da odredimo u kojoj vrsti matrice table se nalazi vrednost b_ind_min
            l=np.where(table[:-1,0]==b_ind_min)[0][0]
            #print(f"Indeks vrste koja izlazi iz baze je l={l+1}, tj. bazna promenljiva {int(b_ind_min)}.")

            # Pivotiranje
            pivot = table[l, j]
            table[l, 1:] = table[l, 1:] / pivot  # Normalizujemo pivot red (osim kolone 0)
            table[l, j] = 1.0  # Precizno postavimo pivot na 1 (u slučaju zaokruživanja)
            table[l, 0] = j     # Ažuriramo baznu promenljivu

            for k in range(m):
                if k != l:
                    faktor = table[k, j]
                    table[k, 1:] -= faktor * table[l, 1:]
                    table[k, j] = 0.0  # Precizno postavimo nulu zbog stabilnosti
        else:
            opt = 2  # Problem je neograničen

    return table, opt

Testirajmo

In [18]:
print_table(primer10_T0)
primer10_T1,opt=simp_one_step_blend(primer10_T0)
print_table(primer10_T1)
primer10_T2,opt=simp_one_step_blend(primer10_T1)
print_table(primer10_T2)

,a1,a2,a3,a4,a5,a6,b
x5,-2.000000,-9.0,1.000000,9.0,1.0,0.0,0.0
x6,0.333333,1.0,-0.333333,-2.0,0.0,1.0,0.0
,-2.000000,-3.0,1.000000,12.0,0.0,0.0,0.0


,a1,a2,a3,a4,a5,a6,b
x5,0.0,-3.0,-1.0,-3.0,1.0,6.0,0.0
x1,1.0,3.0,-1.0,-6.0,0.0,3.0,0.0
,0.0,3.0,-1.0,0.0,0.0,6.0,0.0


,a1,a2,a3,a4,a5,a6,b
x5,0.0,-3.0,-1.0,-3.0,1.0,6.0,0.0
x1,1.0,3.0,-1.0,-6.0,0.0,3.0,0.0
,0.0,3.0,-1.0,0.0,0.0,6.0,0.0


Na kraju kreirajmo funkciju koja iterativno primenjuje simpleks metodu.

In [19]:
def tablicni_simpleks_blend(table, stampaj=0, maks_iter=1000):
    optimalno = 0
    iter = 0

    while optimalno == 0:
        if stampaj == 1:
            print(f"T{iter}")
            print_table(table)

        # Jedna iteracija simpleks algoritma
        table, optimalno = simp_one_step_blend(table)
        iter += 1

        # Prekid ako broj iteracija premaši dozvoljeni maksimum
        if iter > maks_iter:
            print(f"Stop. Broj iteracija je premašio maksimalan broj {maks_iter}")
            break

    # Nakon izlaska iz petlje: štampa rešenja
    if optimalno == 1:
        print(f"\nOptimalna vrednost problema je {-1 * table[-1, -1]:.4f}, a postiže se za:")
        m, n = table.shape
        for k in range(m - 1):
            print(f"x{int(table[k, 0])} = {table[k, -1]:.4f},")
        print("Ostale promenljive imaju vrednost jednaku 0.")
        print(f"Algoritam se završio u {iter - 1}. iteraciji.")

    elif optimalno == 2:
        print("Problem je neograničen.")

    # Završna tabela
    print_table(table)


In [20]:
tablicni_simpleks_blend(primer10_T0,0,6)

Problem je neograničen.


,a1,a2,a3,a4,a5,a6,b
x5,0.0,-3.0,-1.0,-3.0,1.0,6.0,0.0
x1,1.0,3.0,-1.0,-6.0,0.0,3.0,0.0
,0.0,3.0,-1.0,0.0,0.0,6.0,0.0
